In [1]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

In [2]:
import pandas as pd
import datasets.arrow_dataset as da

In [6]:
import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

pretrained_model_name = 'klue/roberta-base'
sts_num_epochs = 4
train_batch_size = 32

sts_model_save_path = 'output/training_sts-'+pretrained_model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [7]:
data1 = da.Dataset.from_pandas(pd.read_csv('./KorSTS/sts-train.tsv', sep='\\t'))
data2 = da.Dataset.from_pandas(pd.read_csv('./KorSTS/sts-dev.tsv', sep='\\t'))

/tmp/ipykernel_12635/1819578977.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data1 = da.Dataset.from_pandas(pd.read_csv('./KorSTS/sts-train.tsv', sep='\\t'))
/tmp/ipykernel_12635/1819578977.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data2 = da.Dataset.from_pandas(pd.read_csv('./KorSTS/sts-dev.tsv', sep='\\t'))


In [8]:
def make_sts_input_example(dataset):
    ''' 
    Transform to InputExample
    ''' 
    input_examples = []
    for i, data in enumerate(dataset):
        sentence1 = data['sentence1']
        sentence2 = data['sentence2']
        score = data['score'] / 5.0  # normalize 0 to 5
        input_examples.append(InputExample(texts=[sentence1, sentence2], label=score))

    return input_examples

In [9]:
sts_train_examples = make_sts_input_example(data1)
sts_valid_examples = make_sts_input_example(data2)

In [10]:
# Train Dataloader
train_dataloader = DataLoader(
    sts_train_examples,
    shuffle=True,
    batch_size=train_batch_size,
)

# Evaluator by sts-validation
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_valid_examples,
    name="sts-dev",
)

In [11]:
pretrained_model_name = 'klue/roberta-base'

# Load Embedding Model
embedding_model = models.Transformer(
    model_name_or_path=pretrained_model_name, 
    max_seq_length=256,
    do_lower_case=True
)

# Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [14]:
# Use CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)

# warmup steps
warmup_steps = math.ceil(len(sts_train_examples) * sts_num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_valid_examples,
    name="sts-dev",
)

# Training
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=dev_evaluator,
    epochs=sts_num_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path=sts_model_save_path
)

2022-05-23 22:45:29 - Warmup-steps: 72



2022-05-23 22:46:01 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 0:
2022-05-23 22:46:03 - Cosine-Similarity :	Pearson: 0.8611	Spearman: 0.8588
2022-05-23 22:46:03 - Manhattan-Distance:	Pearson: 0.8480	Spearman: 0.8500
2022-05-23 22:46:03 - Euclidean-Distance:	Pearson: 0.8478	Spearman: 0.8498
2022-05-23 22:46:03 - Dot-Product-Similarity:	Pearson: 0.8320	Spearman: 0.8275
2022-05-23 22:46:03 - Save model to output/training_sts-klue-roberta-base-2022-05-23_22-43-53



2022-05-23 22:46:38 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 1:
2022-05-23 22:46:40 - Cosine-Similarity :	Pearson: 0.8629	Spearman: 0.8634
2022-05-23 22:46:40 - Manhattan-Distance:	Pearson: 0.8495	Spearman: 0.8539
2022-05-23 22:46:40 - Euclidean-Distance:	Pearson: 0.8493	Spearman: 0.8540
2022-05-23 22:46:40 - Dot-Product-Similarity:	Pearson: 0.8322	Spearman: 0.8320
2022-05-23 22:46:40 - Save model to output/training_sts-klue-roberta-base-2022-05-23_22-43-53



2022-05-23 22:47:14 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 2:
2022-05-23 22:47:17 - Cosine-Similarity :	Pearson: 0.8662	Spearman: 0.8653
2022-05-23 22:47:17 - Manhattan-Distance:	Pearson: 0.8560	Spearman: 0.8591
2022-05-23 22:47:17 - Euclidean-Distance:	Pearson: 0.8561	Spearman: 0.8592
2022-05-23 22:47:17 - Dot-Product-Similarity:	Pearson: 0.8382	Spearman: 0.8349
2022-05-23 22:47:17 - Save model to output/training_sts-klue-roberta-base-2022-05-23_22-43-53



2022-05-23 22:47:51 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset after epoch 3:
2022-05-23 22:47:54 - Cosine-Similarity :	Pearson: 0.8660	Spearman: 0.8657
2022-05-23 22:47:54 - Manhattan-Distance:	Pearson: 0.8560	Spearman: 0.8590
2022-05-23 22:47:54 - Euclidean-Distance:	Pearson: 0.8562	Spearman: 0.8593
2022-05-23 22:47:54 - Dot-Product-Similarity:	Pearson: 0.8397	Spearman: 0.8365
2022-05-23 22:47:54 - Save model to output/training_sts-klue-roberta-base-2022-05-23_22-43-53



In [34]:
s1 = data2['sentence1']
s2 = data2['sentence2']

In [57]:
query = model.encode(s1)
answer = model.encode(s2)

In [ ]:
cos_scores = util.pytorch_cos_sim(query, answer)

In [65]:
print(query.shape)
print(answer.shape)

(1500, 768)
(1500, 768)


In [64]:
print(cos_scores.shape)

torch.Size([1500, 1500])


In [61]:
cos_scores[0]

tensor([ 0.9705, -0.0229,  0.1702,  ..., -0.1053,  0.0639,  0.2182])

In [96]:
i = 0
answer_list = []
for data1, data2 in zip(query, answer):

    cos_scores = util.pytorch_cos_sim(data1, data2)
    
    answer_list.append(cos_scores[0])

In [94]:
import numpy as np

In [100]:
sum(answer_list)/len(answer_list)

tensor([0.5380])

In [97]:
np.shape(answer_list)

(1500,)

In [69]:
len(answer_list)

1500

In [74]:
i = 0
answer_list2 = []
for data1, data2 in zip(s1, s2):
    i = i + 1
    se1 = model.encode(data1)
    se2 = model.encode(data2)

    cos_scores = util.pytorch_cos_sim(se1,se2)
    
    answer_list2.append(cos_scores[0])
    
    if i==50:
        break

Exception ignored in: <function tqdm.__del__ at 0x7fcaab4a89d0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/tqdm/std.py", line 1062, in __del__
    self.close()
  File "/opt/conda/lib/python3.8/site-packages/tqdm/notebook.py", line 233, in close
    super(tqdm_notebook, self).close(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/tqdm/std.py", line 1237, in close
    if self.disable:
AttributeError: 'tqdm_notebook' object has no attribute 'disable'


In [86]:
for i in range(50):
    print(answer_list2[i][0].numpy(), answer_list[i][0].numpy())


0.97048336 0.9704833
0.93510765 0.9351076
0.9875079 0.987508
0.56521106 0.56521106
0.79455245 0.7945525
0.73668957 0.73668957
0.9866762 0.98667645
0.732843 0.7328429
0.92706573 0.9270658
0.9365902 0.9365902
0.8603389 0.86033916
0.42372894 0.42372903
0.9792923 0.9792924
0.93789846 0.9378985
0.95346564 0.9534657
-0.03272952 -0.032729506
0.82682997 0.8268298
0.98293746 0.9829373
0.93395233 0.9339521
0.30477858 0.3047786
0.56129265 0.5612929
0.44255614 0.44255626
0.6884054 0.6884055
0.42372894 0.42372912
0.20343518 0.20343518
0.53395134 0.5339511
0.86960953 0.86960953
0.87493265 0.87493265
0.122579664 0.12257959
-0.025310807 -0.025310716
0.8524226 0.8524226
0.976191 0.976191
0.96434474 0.9643449
0.960942 0.96094215
0.8919307 0.8919308
0.7933205 0.7933202
0.9902598 0.99025977
0.98073196 0.98073226
0.9239159 0.923916
0.2595914 0.25959146
0.6033654 0.6033654
0.74712694 0.74712694
0.08175266 0.08175279
0.71215785 0.71215767
0.44791105 0.4479111
0.10319616 0.10319615
-0.077613 -0.07761296
-0.00

In [45]:
output = model.evaluate(evaluator=dev_evaluator)

2022-05-24 07:54:08 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset:
2022-05-24 07:54:10 - Cosine-Similarity :	Pearson: 0.8660	Spearman: 0.8657
2022-05-24 07:54:10 - Manhattan-Distance:	Pearson: 0.8560	Spearman: 0.8590
2022-05-24 07:54:10 - Euclidean-Distance:	Pearson: 0.8562	Spearman: 0.8593
2022-05-24 07:54:10 - Dot-Product-Similarity:	Pearson: 0.8397	Spearman: 0.8365


In [46]:
output

0.8657151872024093

In [43]:
def postprocess_text(preds, labels):
    preds = [model.encode(pred) for pred in preds]
    labels = [model.encode(label) for label in labels]
    print(preds)

In [44]:
postprocess_text(s1, s2)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [31]:
model.evaluate(evaluator = dev_evaluator)

2022-05-24 07:27:09 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset:
2022-05-24 07:27:11 - Cosine-Similarity :	Pearson: 0.8660	Spearman: 0.8657
2022-05-24 07:27:11 - Manhattan-Distance:	Pearson: 0.8560	Spearman: 0.8590
2022-05-24 07:27:11 - Euclidean-Distance:	Pearson: 0.8562	Spearman: 0.8593
2022-05-24 07:27:11 - Dot-Product-Similarity:	Pearson: 0.8397	Spearman: 0.8365


0.8657151872024093

In [19]:
cos_scores = util.pytorch_cos_sim(s1_embed, s2_embed)

In [20]:
cos_scores

tensor([[0.4138]])

In [16]:
data2[179]

{'genre': 'main-captions',
 'filename': 'MSRvid',
 'year': '2012train',
 'id': 376,
 'score': 2.2,
 'sentence1': '남자가 키보드를 연주하고 있다.',
 'sentence2': '한 남자가 기타를 치고 있다.'}

In [22]:
data2[179]['score']/5

0.44000000000000006

In [23]:
torch.save(model, './RDASS.pt')